<a href="https://colab.research.google.com/github/beifa/M5_Forecasting/blob/master/hyperopt_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gc
import pickle
import random
import numpy as np
import pandas as pd
import lightgbm as lgb

PATH = '/kaggle/input/wmt-inc/'
PATH_PLUS = '/kaggle/input/wmt-plus/'
PATH_WEIGHT = '/kaggle/input/weight-wmt/' #d --> 1158 - 1914
PATH_MODEL = ''

PATH_D1 = ''
PATH_D2 = ''
PATH_D3 = ''
PATH_D4 = ''


SEED = 13
random.seed(SEED)
np.random.seed(SEED)
   
    
    
def read_data(store, start = 0):
    
    remove_features = ['id','state_id','store_id', 'd','sales'] 
    
    encode_f = [
        'enc_dept_id_mean',
        'enc_item_id_mean',   
        'enc_item_id_state_id_mean',      
        'enc_item_id_store_id_mean'   
        ]#best var, 0.47034

    
    ewm = [
        'ewm_lags12_v2',
        #'ewm_lags25_v2'
          ]
    
    
    df = pd.concat([
                pd.read_pickle(PATH_D1),
                pd.read_pickle(PATH_D1),
                pd.read_pickle(PATH_D1)],
                axis =1)
    df= df[df.store_id == store]    
    gc.collect()
    
    
    lags = pd.read_pickle(PATH_D2)
    lags = lags[lags.index.isin(df.index)]
    df = pd.concat([df, lags], axis=1)
    del lags
    gc.collect()
    

    encode = pd.read_pickle(PATH_D3)[encode_f]
    encode = encode[encode.index.isin(df.index)]
    df = pd.concat([df, encode], axis=1)
    del encode
    gc.collect()  
    
    test = pd.read_pickle(PATH_D4)[ewm]
    test = test[test.index.isin(df.index)]
    df = pd.concat([df, test], axis=1)
    del test
    gc.collect()
    
    test = pd.read_pickle('../input/test-pred-wmt/predict_steps_ca.pkl')['pred_CA_1']
    test = test[test.index.isin(df.index)]
    df = pd.concat([df, test], axis=1)
    del test
    gc.collect()
    
    test = pd.read_pickle('../input/test-pred-wmt/predict_steps_wi.pkl')['pred_WI_1']
    test = test[test.index.isin(df.index)]
    df = pd.concat([df, test], axis=1)
    del test
    gc.collect()
    
    df['pred_mean_ca_wi'] = (df['pred_CA_1'] + df['pred_WI_1']) /2
    df = df.drop(['pred_CA_1','pred_WI_1'], axis = 1)
    
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d','sales']+features]
    
    #skip row
    df = df[df.d >= start].reset_index(drop=True)
    return df, features

In [ ]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

In [ ]:
store = 'CA_1'
df, features = read_data(store, start = 0)
tr_mask = df['d']<=1913
train_data = lgb.Dataset(df[tr_mask][features], 
                            label=df[tr_mask]['sales'])

In [ ]:
#index for folds
start = [1745, 1801, 1829, 1857, 1885]
split = []
def iteratt(df, st):
    tr_mask = df['d']<= st
    va_mask = (df['d']>(st)) & (df['d']<= (st+28))
    yield (df[tr_mask].index, df[va_mask].index)
    
for st in (start):    
    for i in iteratt(df, st):
        split.append(i)

In [ ]:
split

[(Int64Index([      0,       1,       2,       3,       4,       5,       6,
                    7,       8,       9,
              ...
              4190911, 4190912, 4190913, 4190914, 4190915, 4190916, 4190917,
              4190918, 4190919, 4190920],
             dtype='int64', length=4190921),
  Int64Index([4190921, 4190922, 4190923, 4190924, 4190925, 4190926, 4190927,
              4190928, 4190929, 4190930,
              ...
              4276171, 4276172, 4276173, 4276174, 4276175, 4276176, 4276177,
              4276178, 4276179, 4276180],
             dtype='int64', length=85260)),
 (Int64Index([      0,       1,       2,       3,       4,       5,       6,
                    7,       8,       9,
              ...
              4361449, 4361450, 4361451, 4361452, 4361453, 4361454, 4361455,
              4361456, 4361457, 4361458],
             dtype='int64', length=4361459),
  Int64Index([4361459, 4361460, 4361461, 4361462, 4361463, 4361464, 4361465,
              4361466, 4

In [ ]:
def fin_param(data, num = 100, num_loop = 50, seed=13):
    """
    bost: ary
    ['gbdt', 'dart', 'goss']
    num - num_boost_round in cv
    num_loop - max_evals  
    """  
    def objective(space):
        print('Start......')
        params = {
            'objective': 'tweedie',
            'boosting': 'gbdt',
            'tweedie_variance_power': space['tweedie_variance_power'],
            'metrics': 'rmse',
            'learning_rate': 0.1,
            'bagging_freq': 1,
            'boost_from_average': False,
            'seed': seed,
            'num_leaves': int(space['num_leaves']),
            #'learning_rate': space['learning_rate'],
            'min_data_in_leaf': int(space['min_data_in_leaf']),
            'bagging_fraction': 1, #space['bagging_fraction'],
            'feature_fraction': 0.9, #space['feature_fraction'],
            #'bagging_freq': int(space['bagging_freq']),         
            'max_depth': -1, #int(space['max_depth']),
            #'min_sum_hessian_in_leaf':  space['min_sum_hessian_in_leaf'],
            #'lambda_l1': space['lambda_l1'],
            #'lambda_l2': space['lambda_l2'],
            #'min_gain_to_split': space['min_gain_to_split'],
            #'max_bin': int(space['max_bin']),
            #'cat_l2': space['cat_l2'],
            #'cat_smooth': space['cat_smooth'] 
            'nthread': 4
            }

        print(params) #params['objective' / 'boosting']
        seed_everything(SEED) 
        cv_results = lgb.cv(params,  data, num_boost_round = num, folds = split,
                            stratified=False, verbose_eval = False, seed = seed)  

        score = cv_results['rmse-mean'][-1]
        print(f'Scores: {score}')
        return {'loss': score, 'status':STATUS_OK}


    space = {
        'tweedie_variance_power': hp.quniform('tweedie_variance_power', 1, 1.9, 0.1),
        #'learning_rate' : hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
        #'learning_rate' : hp.loguniform('learning_rate', np.log(0.1), np.log(0.5)),
        'min_data_in_leaf': hp.choice('min_data_in_leaf',[200, 4095, 3049, 1000]),
        #Я рекомендую использовать min_data_in_leaf, и вы можете установить его значение меньше, чем num_total_data / num_leaves.
        'num_leaves': hp.choice('num_leaves',[2047, 100, 3000, 1000]),
        #'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
        'bagging_fraction': hp.quniform('bagging_fraction', 0.7, 1, 0.1), 
        'feature_fraction': hp.quniform('feature_fraction', 0.7, 1, 0.1),
        #Feature_fraction между (0,3 ~ 0,9), его лучшее значение зависит от вашего набора данных.
        #'bagging_freq': hp.quniform('bagging_freq', 0, 5, 1),     
        #'max_depth': hp.quniform('max_depth', -1, 63, 10),
        #'min_sum_hessian_in_leaf':  hp.uniform('min_sum_hessian_in_leaf', 0.00001, 1),

        
        #если ваш набор данных большой, вы можете игнорировать lambda_l1, lambda_l2 и min_gain_to_split. 
        #Если набор данных маленький, вы можете попробовать сравнительно небольшие lambda_l1 и lambda_l2, например, 0,01 или 0,1.
        #'min_gain_to_split': hp.quniform('min_gain_to_split', 0.1, 5, 0.01),
        #'lambda_l1': hp.uniform('lambda_l1', 0.0, 1.0),
        #'lambda_l2': hp.uniform('lambda_l2', 0.0, 1.0),
        
        #'max_bin': hp.quniform('max_bin', 32, 255, 25),
        
        #'cat_l2': hp.quniform('cat_l2', 1, 10, 1),
        #'cat_smooth': hp.quniform('cat_smooth', 1, 10, 1)
       
        }

    trials = Trials()
    

    best = fmin(fn=objective,
                  space=space,
                  algo=tpe.suggest,
                  max_evals= num_loop,
                  trials=trials)

    print(best)
    return best

In [ ]:
%%time
out = fin_param(train_data, num = 100, num_loop = 10, seed = 13)

Start......
{'objective': 'tweedie', 'boosting': 'gbdt', 'tweedie_variance_power': 1.9000000000000001, 'metrics': 'rmse', 'learning_rate': 0.1, 'bagging_freq': 1, 'boost_from_average': False, 'seed': 13, 'num_leaves': 1000, 'min_data_in_leaf': 3049, 'bagging_fraction': 1, 'feature_fraction': 0.9, 'max_depth': -1, 'nthread': 4}
Scores: 2.0292248891688534
Start......
{'objective': 'tweedie', 'boosting': 'gbdt', 'tweedie_variance_power': 1.8, 'metrics': 'rmse', 'learning_rate': 0.1, 'bagging_freq': 1, 'boost_from_average': False, 'seed': 13, 'num_leaves': 100, 'min_data_in_leaf': 200, 'bagging_fraction': 1, 'feature_fraction': 0.9, 'max_depth': -1, 'nthread': 4}
Scores: 2.0512559626960196
Start......
{'objective': 'tweedie', 'boosting': 'gbdt', 'tweedie_variance_power': 1.4000000000000001, 'metrics': 'rmse', 'learning_rate': 0.1, 'bagging_freq': 1, 'boost_from_average': False, 'seed': 13, 'num_leaves': 1000, 'min_data_in_leaf': 200, 'bagging_fraction': 1, 'feature_fraction': 0.9, 'max_dep

In [ ]:
out

{'bagging_fraction': 0.7000000000000001,
 'feature_fraction': 0.8,
 'min_data_in_leaf': 0,
 'num_leaves': 3,
 'tweedie_variance_power': 1.4000000000000001}